In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import main
sp = main.sp

In [7]:
# import scraped song stats from Excel file

xls = pd.ExcelFile('spotify_charts.xlsx')

map = {}
for sheet_name in xls.sheet_names:
    map[sheet_name] = xls.parse(sheet_name)

In [8]:
# function for splitting the list of songs; spotify API only takes 100 at a time 

def split_list(list_name):
    half = len(list_name)//2
    return list_name[:half], list_name[half:]

In [9]:
def add_song_features(list_of_songs):
    # create dataframe for list of first 100 songs
    songs_dataframe = pd.DataFrame(data=list_of_songs)

    # loop through 100 songs and get track ids
    tracks_list=[]
    for url in list_of_songs['url']:
        track_id = url.replace('https://open.spotify.com/track/','')
        tracks_list.append(track_id)

    # create new column in songs dataframe and add track ids
    songs_dataframe['id'] = tracks_list

    # use sp.audio_features method to get track features using spotify API
    features = []
    features.append(sp.audio_features(tracks=songs_dataframe['id']))

    # loop through features found and convert to a list of values
    features_list = []
    for each in features[0]:
        values = list(each.values())
        features_list.append(values)

    # create features dataframe and add values from features_list to it
    features_dataframe = pd.DataFrame(columns=features[0][0].keys(),data=features_list)

    # merge songs_dataframe and features_dataframe into one big dataframe; merge on track id
    final_dataframe = pd.merge(songs_dataframe,features_dataframe,on='id')
    # rename some columns
    final_dataframe = final_dataframe.rename(columns={"id": "track_id","stream":"num_streams"})
    
    return final_dataframe

In [10]:
writer = pd.ExcelWriter('spotify_charts_features.xlsx', engine='xlsxwriter')

for year_month in map:
    first_100, last_100 = split_list(map[year_month])
    x = add_song_features(first_100)
    x.to_excel(writer, sheet_name='first_100_{}'.format(year_month), index = False)
    y = add_song_features(last_100)
    y.to_excel(writer, sheet_name='last_100_{}'.format(year_month), index = False)

writer.save()